In [286]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import texttable as tt
from sklearn import preprocessing
from sklearn import model_selection 
from sklearn import linear_model 
from sklearn import svm
from sklearn import neural_network

# Loading the data

In [287]:
data = pd.read_csv('data/train.csv', index_col='PassengerId')
del data.index.name # lets also remove this row with just the name on it to make things easier later
data_test = pd.read_csv('data/test.csv', index_col='PassengerId')
del data_test.index.name # lets also remove this row with just the name on it to make things easier later

In [288]:
data[0:3]

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


So, what is the data we have to work with:

Survivied is our target variable we wish to predict, a value of 1 means survival, a value of 0 means death.

Pclass is a proxy for socio-economic status the values are as so:
- 1st = Upper
- 2nd = Middle
- 3rd = Lower

SibSp quantifies how many siblings/spouses were aboard with the passenger.

Parch quantifies how many parents/children were aboard with the passenger.

Embarked codifies the port at which the passenger embarked as so:
- C = Cherbourg
- Q = Queenstown
- S = Southampton





# Feature Engineering / data Pre-processing

In order to use many machine learning algorithms we first have to do some preprocessing to get it into an appropriate format. We'll go through the features one at a time.

### Feature 1: Pclass

First lets check for missing data entries

In [289]:
print('Num of missing entries in training data = {}'.format(sum(np.isnan(data.Pclass))))
print('Num of missing entries in test data = {}'.format(sum(np.isnan(data_test.Pclass))))

Num of missing entries in training data = 0
Num of missing entries in test data = 0


There are no missing entries here so we move on.

Pclass is a categorical variable, these need to be encoded into binary variables. This is nessesary because of the way the algorithm interprets numbers. If we have a categorical feature that takes values 0, 1, 2, 3, 4 it assumes the higher numbers are 'better' (e.g. 4>3) even though they are arbitrary encodings, because ultimately it is calculating values/weights/parameters to be multiplied by these feature variables to give a term which enters into the linear regression. One common way to deal with this is one-hot-encoding, where a feature N takes values 0, 1, 2 for example we would generate 3 features which takes binary values 0 or 1. An example is shown below

We have the original feature data:

| Entry        | N          |
| ------------ |:----------:|
| 0            | 1          | 
| 1            | 2          |
| 2            | 0          |
| 3            | 1          |
| 4            | 2          |
| 5            | 0          |

Which when encoded becomes:

| Entry        | N==0       | N==1       | N==2       |
| ------------ |:----------:|:----------:|:----------:|
| 0            | 0          | 1          | 0          | 
| 1            | 0          | 0          | 1          | 
| 2            | 1          | 0          | 0          | 
| 3            | 0          | 1          | 0          | 
| 4            | 0          | 0          | 1          | 
| 5            | 1          | 0          | 0          | 

In [290]:
# on training data

Pclass_lb = preprocessing.LabelBinarizer()
Pclass_one_hot_encoded = Pclass_lb.fit(data['Pclass']) # one-hot encoding
Pclass_one_hot_encoded = Pclass_lb.transform(data['Pclass']) # one-hot encoding

dfOneHot_Encoded = pd.DataFrame(
    Pclass_one_hot_encoded, 
    columns = ["Pclass_"+str(int(i+1)) for i in range(Pclass_one_hot_encoded.shape[1])],
    index=data.index
    ) # we now construct a dataframe out of this one-hot-encoded data

# we now add our one-hot-encoded Embarked features
data = pd.concat([data, dfOneHot_Encoded], axis=1)
del(data['Pclass']) # and delete the original feature

# on testing data

Pclass_one_hot_encoded = Pclass_lb.transform(data_test['Pclass']) # one-hot encoding
dfOneHot_Encoded = pd.DataFrame(
    Pclass_one_hot_encoded, 
    columns = ["Pclass_"+str(int(i+1)) for i in range(Pclass_one_hot_encoded.shape[1])],
    index=data_test.index
    ) # we now construct a dataframe out of this one-hot-encoded data

# we now add our one-hot-encoded Embarked features
data_test = pd.concat([data_test, dfOneHot_Encoded], axis=1)
del(data_test['Pclass']) # and delete the original feature

### Feature 2: Name

We will drop this feature as names are unlikely to effect survivability.

In [291]:
data.drop(labels=['Name'], axis=1, inplace=True)
data_test.drop(labels=['Name'], axis=1, inplace=True)

### Feature 3: Sex
Sex can be encoded easily as a single binary feature as it only has 2 values, male or female.

In [292]:
data.Sex.unique()

array(['male', 'female'], dtype=object)

In [293]:
le_sex = preprocessing.LabelEncoder()
le_sex.fit(data.Sex) # fits a value to each unique integer value of the feature variable sex
data.Sex = le_sex.transform(data.Sex) # transform the data from labels to numeric

data_test.Sex = le_sex.transform(data_test.Sex) # transform the data from labels to numeric

### Feature 4: Age

Lets first check for missing entries

In [294]:
print('Num of missing entries in training data = {}'.format(sum(np.isnan(data.Age))))
print('Num of missing entries in test data = {}'.format(sum(np.isnan(data_test.Age))))

Num of missing entries in training data = 177
Num of missing entries in test data = 86


We have quite a lot of missing age data, lets just try replacing it with the median of the age data we have.

In [295]:
imputer = preprocessing.Imputer(strategy="median", axis=0)
data['Age'] = imputer.fit_transform(data['Age'].values.reshape(-1, 1))
data_test['Age'] = imputer.transform(data_test['Age'].values.reshape(-1, 1))

In [296]:
print('Num of missing entries in training data = {}'.format(sum(np.isnan(data.Age))))
print('Num of missing entries in test data = {}'.format(sum(np.isnan(data_test.Age))))

Num of missing entries in training data = 0
Num of missing entries in test data = 0


Age is our first continous feature. There are 2 approaches one can take to continuous data such as this, one is to leave it as continous data but perform feature scaling (make it a similar scale to other parameters), another is to discretise it into bins. The second approach *can* help the learning algorithm to pick up on general trends, such as young people and old people perhaps (don't know if this is true for this data) being more likely to die. However we don't know for sure which approach is best for this particular data and should test it rather than picking one based on a *hunch* or preconcieved notions of what approach is best.

For now I will leave this as a continous variable and perform feature scaling, later I will try the binning method and compare the results with the different learning algorithms to decide which approach is best for this data.

Feature scaling makes it so the mean is 0 and the variance one, applying this to all continous variables means they are all on the same scale and thus will be weighted similarly and the parameter space can converge in minimising the cost function.

In [297]:
scaler_age = preprocessing.StandardScaler().fit(data['Age'].values.reshape(-1, 1))
data['Age'] = scaler_age.transform(data['Age'].values.reshape(-1, 1))
data_test['Age'] = scaler_age.transform(data_test['Age'].values.reshape(-1, 1))

### Feature 5 & 6: SibSp and Parch

These 2 parameters can be combined to parameterise the number of relatives aboard the ship, which seems like a sensible combination to make.

In [298]:
data['NumRels'] = data['SibSp'] + data['Parch']
data_test['NumRels'] = data_test['SibSp'] + data_test['Parch']
data.drop(labels=['SibSp', 'Parch'], axis=1, inplace=True)
data_test.drop(labels=['SibSp', 'Parch'], axis=1, inplace=True)

In [299]:
print('Num of missing entries in training data = {}'.format(sum(np.isnan(data.NumRels))))
print('Num of missing entries in test data = {}'.format(sum(np.isnan(data_test.NumRels))))

Num of missing entries in training data = 0
Num of missing entries in test data = 0


We will now treat this new feature as a continuous variable rather than treating it as a catagorical variable which takes on discrete values, which is another valid approach. Therefore we perform feature scaling on this new continous variable.

In [300]:
scaler_NumRels = preprocessing.StandardScaler().fit(data['NumRels'].values.reshape(-1, 1))
data['NumRels'] = scaler_NumRels.transform(data['NumRels'].values.reshape(-1, 1))
data_test['NumRels'] = scaler_NumRels.transform(data_test['NumRels'].values.reshape(-1, 1))

/Users/ajs3g11/anaconda/envs/python3/lib/python3.6/site-packages/sklearn/utils/validation.py:444: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


### Feature 7: Ticket

We just delete this information since they are unique ticket numbers and contain no relevant information.

In [301]:
data.drop(labels=['Ticket'], axis=1, inplace=True)
data_test.drop(labels=['Ticket'], axis=1, inplace=True)

### Feature 8: Fare

Fare is another continous feature we want to feature scale, or could discretise alternatively,  here we feature scale and treat it as continous.

In [302]:
print('Num of missing entries in training data = {}'.format(sum(np.isnan(data.Fare))))
print('Num of missing entries in test data = {}'.format(sum(np.isnan(data_test.Fare))))

Num of missing entries in training data = 0
Num of missing entries in test data = 1


However before we do that we need to take care of the missing entry in the testing data. We will use the median of the training data to replace the missing entry.

In [303]:
imputer = preprocessing.Imputer(strategy="median", axis=0)
imputer.fit(data['Fare'].values.reshape(-1, 1))
data_test['Fare'] = imputer.transform(data_test['Fare'].values.reshape(-1, 1))

In [304]:
scaler_Fare = preprocessing.StandardScaler().fit(data['Fare'].values.reshape(-1, 1))
data['Fare'] = scaler_Fare.transform(data['Fare'].values.reshape(-1, 1))
data_test['Fare'] = scaler_Fare.transform(data_test['Fare'].values.reshape(-1, 1))

### Feature 9: Cabin

In [305]:
print('Num of missing entries in training data = {}'.format(sum(pd.isnull(data['Cabin']))))
print('Num of missing entries in test data = {}'.format(sum(pd.isnull(data_test['Cabin']))))

Num of missing entries in training data = 687
Num of missing entries in test data = 327


There may well be some useful information here about the position on the ship of the passanger's cabins but there are so many missing values that we will just delete this feature for now.

In [306]:
data.drop(labels=['Cabin'], axis=1, inplace=True)
data_test.drop(labels=['Cabin'], axis=1, inplace=True)

### Feature 10: Embarked

As with the PClass data we want to encode this categorical feature into many binary features, we once again you one-hot encoding.

In [307]:
print('Num of missing entries in training data = {}'.format(sum(pd.isnull(data.Embarked))))
print('Num of missing entries in test data = {}'.format(sum(pd.isnull(data_test.Embarked))))

Num of missing entries in training data = 2
Num of missing entries in test data = 0


Since there's only 2 null values in the training data we'll drop them.

In [308]:
data[0:5].drop([3, 4])

,Survived,Sex,Age,Fare,Embarked,Pclass_1,Pclass_2,Pclass_3,NumRels
1,0,1,-0.565736,-0.502445,S,0,0,1,0.059160
2,1,0,0.663861,0.786845,C,1,0,0,0.059160
5,0,1,0.433312,-0.486337,S,0,0,1,-0.560975


In [309]:
indexes_of_nulls = [i for i, x in zip(data.index, pd.isnull(data.Embarked)) if x]
print(indexes_of_nulls)
data.drop(np.array(indexes_of_nulls), inplace=True) # remove examples with null values for embarked

[62, 830]


In [310]:
# on training data

Embarked_lb = preprocessing.LabelBinarizer()
Embarked_one_hot_encoded = Embarked_lb.fit(data['Embarked']) # one-hot encoding
Embarked_one_hot_encoded = Embarked_lb.transform(data['Embarked']) # one-hot encoding

dfOneHot_Encoded = pd.DataFrame(
    Embarked_one_hot_encoded, 
    columns = ["Embarked_"+str(int(i)) for i in range(Embarked_one_hot_encoded.shape[1])],
    index=data.index
    ) # we now construct a dataframe out of this one-hot-encoded data

# we now add our one-hot-encoded Embarked features
data = pd.concat([data, dfOneHot_Encoded], axis=1)
del(data['Embarked']) # and delete the original feature

# on testing data

Embarked_one_hot_encoded = Embarked_lb.transform(data_test['Embarked']) # one-hot encoding
dfOneHot_Encoded = pd.DataFrame(
    Embarked_one_hot_encoded, 
    columns = ["Embarked_"+str(int(i)) for i in range(Embarked_one_hot_encoded.shape[1])],
    index=data_test.index
    ) # we now construct a dataframe out of this one-hot-encoded data

# we now add our one-hot-encoded Embarked features
data_test = pd.concat([data_test, dfOneHot_Encoded], axis=1)
del(data_test['Embarked']) # and delete the original feature

In [311]:
data[0:5]

,Survived,Sex,Age,Fare,Pclass_1,Pclass_2,Pclass_3,NumRels,Embarked_0,Embarked_1,Embarked_2
1,0,1,-0.565736,-0.502445,0,0,1,0.059160,0,0,1
2,1,0,0.663861,0.786845,1,0,0,0.059160,1,0,0
3,1,0,-0.258337,-0.488854,0,0,1,-0.560975,0,0,1
4,1,0,0.433312,0.420730,1,0,0,0.059160,0,0,1
5,0,1,0.433312,-0.486337,0,0,1,-0.560975,0,0,1


#### We can now seperate our feature and target variables for the training data and begin to train some machine learning classifiers!

In [322]:
X = data.drop(["Survived"], axis=1)
y = data["Survived"]

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.10) # we will also hold back 10% of our data as test data to appraise the fitted and tuned models after validation before applying them to our Kaggle test data

### Validation Methods

Before we start training models it's important to consider how we will assess the accuracy of the predictions it makes. Assessing this on the data we used to train it is a very bad practise and is called the *resubstitution error*. This is because you cannot know if you are overfitting or not if you assess your model on the training data.

So how do we assess the quality of our learning algorithms? There are a few approaches called Validation (because they validate our model). It is also important to have a 3rd set of test data. The purpose of this is that we train many learning algorithms on the trianing data and validate the performance of them on the validation data to pick which one to use. The performance of the algorithm on the validation data is no-longer a fair assessment of the accuracy, as the algorithm was picked for the fact it performed well on that data. We now assess the accuracy of the model fitted with training data and tuned (as in, we pick the algorithm we are using and the hyperparameters for that algorithm) by validation data on data it has never seen, i.e. test data. This should give us a fair assessment of the actual accuracy of the algorithm.

#### Holdout
The first approach is simple, we simply split the data into a training set and validation (also called cross-validation) set, also called the holdout set (as we hold it out from the training set). We train the model on the training data and validate the model on the valdiation data. In this case it is important to be careful that the sets contain similar occurances of different classes in the data, this process of equal distribution of classes is called *stratification*.

#### K-Fold Cross-Validation
The problem with this first approach is that we lost part of the training data we'd like our model to learn from, by reducing the data in this way we are forced to use a lower bias model (more data => higher variance models can be fit => higher accuracy can be achieved).

In K-Fold cross validation the data is divided into k subsets and we perform the hold-out method k times, such that each time, one of the k subsets is used as the validation set and the other k-1 subsets are put together to form a training set. The error estimation is averaged over all k trials to get total effectiveness of our model. In this scheme every data point gets to be in a validation set exactly once, and every data point gets to be in a training set k-1 times. 

#### Stratified K-Fold Cross Validation
This is the same as above but care is taken such that each fold contains approximately the same percentage of samples of each target class as the complete set, or in case of prediction problems, the mean response value is approximately equal in all the folds.

#### Leave-$P$-out Cross Validation (or Leave-One-Out Cross Validation [LOOCV])
This approach is similar to k-folds cross validation. We leave $P$ points out of the training data and validate with these p points. We then repeat this for all possible combinations. Commonly $P$ is set to $1$ and this becomes ***Leave-one-out Cross Validation***. This is generally preferred as it is easy to compute all combinations (there are $m$, where m = number of training examples)


## Logistic Regression

In [314]:
# We now use scikit learn to fit a regularised logistic regression 
# model with each of the feature variables being linear
# setting fit_intercept=True fits the Theta_0 term - an intercept term

lr_model = linear_model.LogisticRegression(fit_intercept=True)


In [345]:
# lets use stratified K-folds cross validation with 10 folds to evaluate our algorithm - can use sklearn.model_selection.cross_val_score for this

In [348]:
N_splits = 10
kfold = model_selection.KFold(n_splits=N_splits)

scoring = 'accuracy'
score = model_selection.cross_val_score(lr_model, X_train, y_train, cv=kfold, n_jobs=1, scoring=scoring)


In [351]:
score.mean() # we get an average accuracy of 80% over 10 folds with logistic regression

0.80249999999999999

In [352]:
# we now train our model, which we've confirmed should give ~80% accuracy, on our entire training data set 
lr_model.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [354]:
# and test it on our test data we have held back from the training/validation set
lr_model.score(X_test, y_test) 

0.797752808988764

We also get ~80% accuracy, so this suggests the model is fitted correctly and is performing well, we now predict on Kaggle's test data and save the prediction to submit to Kaggle.

In [355]:
prediction = lr_model.predict(data_test)

In [361]:
with open('prediction_submission_LR.csv', 'w') as file:
    print('PassengerId,Survived', file=file)
    for i, id_ in enumerate(data_test.index):
        print('{},{}'.format(id_, prediction[i]), file=file)


When we submit this result to Kaggle we get a reported accuracy of 76.076% which is significantly lower than we evaluated on our held back test data, why?